In [2]:
# Cell 1 — Install and Imports
import csv
import os
from pathlib import Path
import cv2
import pandas as pd
from ultralytics import YOLO
import numpy as np

BASE_DIR = str(Path(os.getcwd()).parent)

DATASET_DIR   = os.path.join(BASE_DIR, "hip_dataset")
RAW_XRAY_DIR  = os.path.join(BASE_DIR, "raw_xrays")
PROCESSED_DIR = os.path.join(BASE_DIR, "data", "processed_hips")
YOLO_CONFIG   = os.path.join(BASE_DIR, "hip_dataset.yaml")

os.makedirs(PROCESSED_DIR, exist_ok=True)

print("✅ BASE_DIR:", BASE_DIR)
print("✅ DATASET_DIR:", DATASET_DIR)
print("✅ RAW_XRAY_DIR:", RAW_XRAY_DIR)
print("✅ PROCESSED_DIR:", PROCESSED_DIR)
print("✅ YOLO_CONFIG:", YOLO_CONFIG)


✅ BASE_DIR: /Users/aryan078/Desktop/CHD_project
✅ DATASET_DIR: /Users/aryan078/Desktop/CHD_project/hip_dataset
✅ RAW_XRAY_DIR: /Users/aryan078/Desktop/CHD_project/raw_xrays
✅ PROCESSED_DIR: /Users/aryan078/Desktop/CHD_project/data/processed_hips
✅ YOLO_CONFIG: /Users/aryan078/Desktop/CHD_project/hip_dataset.yaml


In [ ]:
with open(YOLO_CONFIG, "w") as f:
    f.write(f"""
train: {os.path.join(DATASET_DIR, 'images/train')}
val: {os.path.join(DATASET_DIR, 'images/val')}

nc: 1
names: ['hip']
""")

print("YOLO config file written.")

YOLO config file written.


In [ ]:
# Cell 3 — High Recall YOLOv8 Training
model = YOLO("yolov8m.pt")  # More capacity than yolov8n, better recall

model.train(
    data=YOLO_CONFIG,       
    epochs=150,             # Enough epochs for small set; avoid extreme overfit
    imgsz=512,
    batch=4,                # preferred batch size
    optimizer='Adam',       # Better on small sets
    lr0=0.001,               # Slightly lower than default
    val=True,               # Keep val split for monitoring
    
    # Strong augmentation for better generalization
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=20,
    translate=0.1,
    scale=0.5,
    shear=5.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.2,
    
    project=os.path.join(BASE_DIR, "runs", "detect"),
    name="hip_singleclass_max_recall"
)


New https://pypi.org/project/ultralytics/8.3.182 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.180 🚀 Python-3.12.8 torch-2.2.2 CPU (Intel Core(TM) i5-7360U 2.30GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/aryan078/Desktop/CHD_project/hip_dataset.yaml, degrees=20, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=hip_singleclass_max_recal

train: Scanning /Users/aryan078/Desktop/CHD_project/hip_dataset/labels/train.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]

albumentations: ImageCompression.__init__() got an unexpected keyword argument 'quality_range'
val: Fast image access ✅ (ping: 0.1±0.1 ms, read: 404.4±448.1 MB/s, size: 912.0 KB)



val: Scanning /Users/aryan078/Desktop/CHD_project/hip_dataset/labels/val.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]

Plotting labels to /Users/aryan078/Desktop/CHD_project/runs/detect/hip_singleclass_max_recall/labels.jpg... 


...

                   all          5          5      0.991        0.8      0.838      0.388



150 epochs completed in 1.725 hours.
Optimizer stripped from /Users/aryan078/Desktop/CHD_project/runs/detect/hip_singleclass_max_recall/weights/last.pt, 52.0MB
Optimizer stripped from /Users/aryan078/Desktop/CHD_project/runs/detect/hip_singleclass_max_recall/weights/best.pt, 52.0MB

Validating /Users/aryan078/Desktop/CHD_project/runs/detect/hip_singleclass_max_recall/weights/best.pt...
Ultralytics 8.3.180 🚀 Python-3.12.8 torch-2.2.2 CPU (Intel Core(TM) i5-7360U 2.30GHz)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<00:00,  4.29s/it]


                   all          5          5          1      0.798      0.799      0.431
Speed: 1.1ms preprocess, 766.5ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to /Users/aryan078/Desktop/CHD_project/runs/detect/hip_singleclass_max_recall


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x1577238c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

In [ ]:
# Cell 4 — Inference on raw_xrays
trained_model_path = os.path.join(BASE_DIR, "runs", "detect", "hip_singleclass_max_recall", "weights", "best.pt")
model = YOLO(trained_model_path)

results = model.predict(
    source=RAW_XRAY_DIR,
    imgsz=512,
    conf=0.35,
    iou=0.6,
    augment=True,
    save=False,
    save_txt=False,
    verbose=False,  # hides YOLO's raw detections log
    project=os.path.join(BASE_DIR, "runs", "detect"),
    name="hip_predictions_tta"
)


# Output dirs
save_dir = Path(BASE_DIR) / "runs" / "detect" / "hip_predictions_clean"
label_dir = save_dir / "labels"
img_dir = save_dir / "images"

label_dir.mkdir(parents=True, exist_ok=True)
img_dir.mkdir(parents=True, exist_ok=True)

modified_count = 0

for r in results:
    # Keep only top-1 by confidence
    if len(r.boxes) > 0:
        confs = r.boxes.conf.cpu().numpy()
        idx_max = confs.argmax()
        r.boxes = r.boxes[idx_max:idx_max+1]
        modified_count += 1

    # Save TXT
    label_path = label_dir / (Path(r.path).stem + ".txt")
    if len(r.boxes) > 0:
        cls = int(r.boxes.cls[0].item())
        xywh = r.boxes.xywhn[0].cpu().numpy()
        conf = r.boxes.conf[0].item()
        with open(label_path, "w") as f:
            f.write(f"{cls} {xywh[0]} {xywh[1]} {xywh[2]} {xywh[3]} {conf:.4f}")
    else:
        open(label_path, "w").close()

    # Save filtered preview image
    im_array = r.plot()
    cv2.imwrite(str(img_dir / Path(r.path).name), im_array)

print(f"✅ Saved predictions in: {save_dir}")
print(f"🔹 Enforced exactly one hip per image for {modified_count} images.")


✅ Saved predictions in: /Users/aryan078/Desktop/CHD_project/runs/detect/hip_predictions_clean
🔹 Enforced exactly one hip per image for 99 images.


In [ ]:
#using the YOLO model to crop first 100 images
save_crops_dir = Path(BASE_DIR) / "data" / "processed_hips"
save_crops_dir.mkdir(parents=True, exist_ok=True)

csv_path = save_crops_dir / "crop_info.csv"

# Open CSV for writing
with open(csv_path, mode="w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        "original_image", "hip_class", "confidence", 
        "x_min", "y_min", "width", "height", "crop_filename"
    ])

    for r in results:
        img_path = Path(r.path)
        img = cv2.imread(str(img_path))
        h_img, w_img = img.shape[:2]

        if len(r.boxes) == 0:
            continue

        # YOLO normalized center x/y, w/h
        cx, cy, bw, bh = r.boxes.xywhn[0].cpu().numpy()
        conf = float(r.boxes.conf[0].item())
        cls_id = int(r.boxes.cls[0].item())

        # Convert to pixel coords
        cx_pix, cy_pix = cx * w_img, cy * h_img
        bw_pix, bh_pix = bw * w_img, bh * h_img

        x1 = cx_pix - bw_pix / 2
        y1 = cy_pix - bh_pix / 2
        x2 = cx_pix + bw_pix / 2
        y2 = cy_pix + bh_pix / 2

        # Expand box by padding factor
        pad_factor = 0.4  # 40% padding
        dw = bw_pix * pad_factor
        dh = bh_pix * pad_factor
        x1 -= dw
        y1 -= dh
        x2 += dw
        y2 += dh

        # Make box square
        box_w = x2 - x1
        box_h = y2 - y1
        side = max(box_w, box_h)
        cx_box = (x1 + x2) / 2
        cy_box = (y1 + y2) / 2
        x1 = cx_box - side / 2
        x2 = cx_box + side / 2
        y1 = cy_box - side / 2
        y2 = cy_box + side / 2

        # Clip to image boundaries
        x1 = max(0, int(x1))
        y1 = max(0, int(y1))
        x2 = min(w_img, int(x2))
        y2 = min(h_img, int(y2))

        # Crop hip ROI
        crop = img[y1:y2, x1:x2]

        # Resize to 512×512 for ResNet
        crop_resized = cv2.resize(crop, (512, 512), interpolation=cv2.INTER_AREA)

        # Save crop
        crop_filename = f"{img_path.stem}_hip{cls_id}.jpg"
        cv2.imwrite(str(save_crops_dir / crop_filename), crop_resized)

        # Write CSV row
        writer.writerow([
            img_path.name, cls_id, conf, 
            x1, y1, (x2-x1), (y2-y1), crop_filename
        ])

print(f"✅ Crops saved to: {save_crops_dir}")
print(f"📄 Crop info CSV saved to: {csv_path}")


✅ Crops saved to: /Users/aryan078/Desktop/CHD_project/data/processed_hips
📄 Crop info CSV saved to: /Users/aryan078/Desktop/CHD_project/data/processed_hips/crop_info.csv


In [ ]:
#preprocessing the cropped images
crops_dir = Path(BASE_DIR) / "data" / "processed_hips"
preproc_dir = Path(BASE_DIR) / "data" / "processed_hips_preproc_sharp"
preproc_dir.mkdir(parents=True, exist_ok=True)

for crop_path in crops_dir.glob("*.jpg"):
    # Read grayscale
    img = cv2.imread(str(crop_path), cv2.IMREAD_GRAYSCALE)

    # Single resize to target size if needed
    img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_CUBIC)

    # CLAHE for local contrast with mild clip limit
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_clahe = clahe.apply(img)
    
    # Mild denoise skip Bilateral or use very low params
    # img_denoised = cv2.bilateralFilter(img_clahe, d=3, sigmaColor=20, sigmaSpace=20)
    img_denoised = img_clahe  # No smoothing to keep sharpness

    # Stronger unsharp masking
    gaussian_blur = cv2.GaussianBlur(img_denoised, (0,0), sigmaX=0.5)
    img_sharp = cv2.addWeighted(img_denoised, 1.8, gaussian_blur, -0.8, 0)

    # Laplacian edge boost
    lap = cv2.Laplacian(img_sharp, cv2.CV_16S, ksize=3)
    lap = cv2.convertScaleAbs(lap)
    img_final = cv2.addWeighted(img_sharp, 1.0, lap, 0.3, 0)  # blend edges

    # Convert to 3‑channel
    img_rgb = cv2.cvtColor(img_final, cv2.COLOR_GRAY2BGR)

    out_path = preproc_dir / crop_path.name
    cv2.imwrite(str(out_path), img_rgb)

print(f"✅ High‑sharpness preprocessed crops saved to {preproc_dir}")


✅ High‑sharpness preprocessed crops saved to /Users/aryan078/Desktop/CHD_project/data/processed_hips_preproc_sharp


In [3]:
# doing the same thing for second 100 images
# CONFIG
BASE_DIR = Path(BASE_DIR)  # convert to Path if it was originally a string
# Folder with your new 333 raw DICOM/JPG images
RAW_XRAY_DIR_110 = BASE_DIR / "data" / "raw" / "110_raw"

# Where the new crops + preprocessed images will go
save_crops_dir_110 = BASE_DIR / "data" / "processed_hips_batch_110"
preproc_dir_110    = BASE_DIR / "data" / "processed_hips_preproc_sharp_batch_110"
csv_path_110       = save_crops_dir_110 / "crop_info_batch_110.csv"

# YOLO model weights (same as before)
YOLO_WEIGHTS = BASE_DIR / "runs" / "detect" / "hip_singleclass_max_recall" / "weights" / "best.pt"

# Create directories
save_crops_dir_110.mkdir(parents=True, exist_ok=True)
preproc_dir_110.mkdir(parents=True, exist_ok=True)


In [4]:
# 1. YOLO Inference
model = YOLO(str(YOLO_WEIGHTS))
results = model.predict(
    source=str(RAW_XRAY_DIR_110),
    imgsz=512,
    conf=0.35,
    iou=0.6,
    augment=True,
    save=False,
    save_txt=False,
    verbose=False
)

In [5]:
# 2. CROP & SAVE
with open(csv_path_110, mode="w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        "original_image", "hip_class", "confidence",
        "x_min", "y_min", "width", "height", "crop_filename"
    ])

    total_crops = 0

    for r in results:
        img_path = Path(r.path)
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"Could not read: {img_path}")
            continue

        h_img, w_img = img.shape[:2]
        if len(r.boxes) == 0:
            print(f"No detection in: {img_path.name}")
            continue

        # Take the highest confidence box
        idx_max = r.boxes.conf.cpu().numpy().argmax()
        r.boxes = r.boxes[idx_max:idx_max+1]

        cx, cy, bw, bh = r.boxes.xywhn[0].cpu().numpy()
        conf = float(r.boxes.conf[0].item())
        cls_id = int(r.boxes.cls[0].item())

        cx_pix, cy_pix = cx * w_img, cy * h_img
        bw_pix, bh_pix = bw * w_img, bh * h_img

        x1 = cx_pix - bw_pix / 2
        y1 = cy_pix - bh_pix / 2
        x2 = cx_pix + bw_pix / 2
        y2 = cy_pix + bh_pix / 2

        # Padding
        pad_factor = 0.4
        dw, dh = bw_pix * pad_factor, bh_pix * pad_factor
        x1 -= dw
        y1 -= dh
        x2 += dw
        y2 += dh

        # Make square
        box_w, box_h = x2 - x1, y2 - y1
        side = max(box_w, box_h)
        cx_box = (x1 + x2) / 2
        cy_box = (y1 + y2) / 2
        x1 = cx_box - side / 2
        x2 = cx_box + side / 2
        y1 = cy_box - side / 2
        y2 = cy_box + side / 2

        # Clip to image boundaries
        x1 = max(0, int(x1))
        y1 = max(0, int(y1))
        x2 = min(w_img, int(x2))
        y2 = min(h_img, int(y2))

        # Crop & resize
        crop = img[y1:y2, x1:x2]
        if crop.shape[0] == 0 or crop.shape[1] == 0:
            print(f"Empty crop for {img_path.name}")
            continue
        crop_resized = cv2.resize(crop, (512, 512), interpolation=cv2.INTER_AREA)

        crop_filename = f"{img_path.stem}_hip{cls_id}.jpg"
        cv2.imwrite(str(save_crops_dir_110 / crop_filename), crop_resized)

        writer.writerow([
            img_path.name, cls_id, conf,
            x1, y1, (x2 - x1), (y2 - y1), crop_filename
        ])

        total_crops += 1

print(f"✅ Saved {total_crops} crops for batch_110 to: {save_crops_dir_110}")
print(f" Crop info CSV: {csv_path_110}")

# 3. PREPROCESSING
for crop_path in save_crops_dir_110.glob("*.jpg"):
    img = cv2.imread(str(crop_path), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_CUBIC)

    # CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_clahe = clahe.apply(img)

    # Unsharp masking
    gaussian_blur = cv2.GaussianBlur(img_clahe, (0, 0), sigmaX=0.5)
    img_sharp = cv2.addWeighted(img_clahe, 1.8, gaussian_blur, -0.8, 0)

    # Laplacian sharpening
    lap = cv2.Laplacian(img_sharp, cv2.CV_16S, ksize=3)
    lap = cv2.convertScaleAbs(lap)
    img_final = cv2.addWeighted(img_sharp, 1.0, lap, 0.3, 0)

    # Convert back to RGB
    img_rgb = cv2.cvtColor(img_final, cv2.COLOR_GRAY2BGR)
    cv2.imwrite(str(preproc_dir_110 / crop_path.name), img_rgb)

print(f"✅ Preprocessed images for batch_110 saved to: {preproc_dir_110}")

No detection in: Radiograph3 copy 13.jpg
No detection in: Radiograph4 copy 6.jpg
✅ Saved 119 crops for batch_110 to: /Users/aryan078/Desktop/CHD_project/data/processed_hips_batch_110
 Crop info CSV: /Users/aryan078/Desktop/CHD_project/data/processed_hips_batch_110/crop_info_batch_110.csv
✅ Preprocessed images for batch_110 saved to: /Users/aryan078/Desktop/CHD_project/data/processed_hips_preproc_sharp_batch_110
